In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
import plotly

import time
from datetime import datetime
import datetime as dt

import mysql.connector
from sqlalchemy import create_engine

# for decoding the downloaded files
import io

In [ ]:
# Making API call to PushShift API
import requests

In [ ]:
query = "seo"
url = f"https://api.pushshift.io/reddit/search/comment/?q={query}"
request = requests.get(url)
json_response = request.json()
json_response

In [ ]:
type(json_response)

In [ ]:
def get_pushshift_data(data_type, **kwargs):
    """
    Gets data from the pushshift api.
 
    data_type can be 'comment' or 'submission'
    The rest of the args are interpreted as payload.
 
    Read more: https://github.com/pushshift/api
    """
 
    base_url = f"https://api.pushshift.io/reddit/search/{data_type}/"
    payload = kwargs
    request = requests.get(base_url, params=payload)
    return request.json()

In [ ]:
data_type="comment"     # give me comments, use "submission" to publish something
query="python"          # Add your query
duration="30d"          # Select the timeframe. Epoch value or Integer + "s,m,h,d" (i.e. "second", "minute", "hour", "day")
size=1000               # maximum 1000 comments
sort_type="score"       # Sort by score (Accepted: "score", "num_comments", "created_utc")
sort="desc"             # sort descending
aggs="subreddit"        #"author", "link_id", "created_utc", "subreddit"

In [ ]:
data = get_pushshift_data(data_type=data_type,
                          q=query,
                          after=duration,
                          size=size,
                          aggs=aggs)

In [ ]:
data

In [ ]:
data_df = pd.DataFrame(data['data'])

In [ ]:
data_df

In [ ]:
data_grouped = data_df.groupby("subreddit_name_prefixed").agg({"subreddit_id":"count"})

In [ ]:
data_grouped = data_grouped.sort_values("subreddit_id", ascending=False)

In [ ]:
data_grouped = data_grouped.reset_index()

In [ ]:
data_grouped

In [ ]:
# matplotlib
plt.figure(figsize=(16,9))

plt.bar(data_grouped['subreddit_name_prefixed'][0:10], data_grouped['subreddit_id'][0:10])
plt.xticks(rotation=45)
plt.ylabel("Frequency of comments")
plt.xlabel("Subreddits")
plt.show()

In [ ]:
import plotly.express as px
 
px.bar(data_grouped,              # our dataframe
       x="subreddit_name_prefixed",         # x will be the 'key' column of the dataframe
       y="subreddit_id",   # y will be the 'doc_count' column of the dataframe
       title=f'Subreddits with most activity - comments with "{query}" in the last "{duration}"',
       labels={"doc_count": "# comments","key": "Subreddits"}, # the axis names
       color_discrete_sequence=["#1f77b4"], # the colors used
       height=500,
       width=800)

In [ ]:
data_type = "comment"
query_word = "VIAC"
aggs = "subreddit"
after = 1
subreddit = "wallstreetbets"
sort = "desc"

In [ ]:
# function to query last 1000 comments from any subreddit

def extract_comments(data_type, **kwargs):
    base_url = f"https://api.pushshift.io/reddit/search/{data_type}"
    payload = kwargs
    request = requests.get(base_url, params=payload)
    return request.url, request.json()

In [ ]:
comms = extract_comments(data_type = data_type,
                  subreddit=subreddit,
                  aggs=aggs,
                  sort=sort,
                  after="1d",
                  size=500)

In [ ]:
comms2 = extract_comments(data_type = data_type,
                  subreddit=subreddit,
                  aggs=aggs,
                  sort=sort,
                  after="1d",
                  size=400)

In [ ]:
comms2_df = pd.DataFrame(comms2[1]['data'])
comms2_df

In [ ]:
comms[0]

In [ ]:
comms[1]

In [ ]:
comms_df = pd.DataFrame(comms[1]['data'])

In [ ]:
comms_df = comms_df[['author', 'body', 'subreddit', 'subreddit_name_prefixed', 'permalink', 'created_utc']]

In [ ]:
comms_df

In [ ]:
def utc_to_date_obj(date):
    str_date = time.strftime("%Y-%m-%d", time.localtime(date))
    date_obj = datetime.strptime(str_date, "%Y-%m-%d").date()
    return date_obj

In [ ]:
comms_df['created_utc2'] = comms_df['created_utc'].apply(lambda x: utc_to_date_obj(x))

In [ ]:
comms_df.drop(labels=['created_utc'], axis=1, inplace=True)

In [ ]:
comms_df.rename(columns={"created_utc2":"timestamp"}, inplace=True)

In [ ]:
comms_df

In [ ]:
db = 'wsb_data'
db_tbl_name = 'wsb_comments'

'''
Create a mapping of df dtypes to mysql data types (not perfect, but close enough)
'''
def dtype_mapping():
    return {'object' : 'TEXT',
        'int64' : 'INT',
        'float64' : 'FLOAT',
        'datetime64' : 'DATETIME',
        'bool' : 'TINYINT',
        'category' : 'TEXT',
        'timedelta[ns]' : 'TEXT'}

'''
Create a sqlalchemy engine
'''
def mysql_engine(user = 'root', password = 'Blackstar5140!', host = '127.0.0.1', port = '3306', database = db):
    engine = create_engine("mysql://{0}:{1}@{2}:{3}/{4}?charset=utf8mb4".format(user, password, host, port, database))
    return engine

'''
Create a mysql connection from sqlalchemy engine
'''
def mysql_conn(engine):
    conn = engine.raw_connection()
    return conn

'''
Create sql input for table names and types
'''
def gen_tbl_cols_sql(df):
    dmap = dtype_mapping()
    sql = "comment_id INT AUTO_INCREMENT PRIMARY KEY"
    df1 = df.rename(columns = {"" : "nocolname"})
    hdrs = df1.dtypes.index
    hdrs_list = [(hdr, str(df1[hdr].dtype)) for hdr in hdrs]
    for i, hl in enumerate(hdrs_list):
        sql += " ,{0} {1}".format(hl[0], dmap[hl[1]])
    return sql

'''
Create a mysql table from a df
'''
def create_mysql_tbl_schema(df, conn, db, tbl_name):
    tbl_cols_sql = gen_tbl_cols_sql(df)
    sql = "USE {0}; CREATE TABLE {1} ({2})".format(db, tbl_name, tbl_cols_sql)
    cur = conn.cursor()
    cur.execute(sql)
    cur.close()
    conn.commit()

'''
Write df data to newly create mysql table
'''
def df_to_mysql(df, engine, tbl_name):
    df.to_sql(tbl_name, engine, if_exists='append', method='multi')

# df = comms_df.copy()
# # create_mysql_tbl_schema(df, mysql_conn(mysql_engine()), db, db_tbl_name)
# df_to_mysql(df, mysql_engine(), db_tbl_name)

In [ ]:
sql_engine = mysql_engine()
sql_connection = sql_engine.connect()
comms_df.to_sql(db_tbl_name, sql_engine, if_exists='append', method='multi')

In [ ]:
from sqlalchemy import text

In [ ]:
# Extract the data from the MySQL database
# SQL Code to run
sql_code = text("""
select * 
from wsb_data.wsb_comments;
""")

# creating an engine with attributes provided
sql_engine = mysql_engine()
# establishing connection with the database
sql_connection = sql_engine.connect()
# Executing the sql code.
extracted_data = sql_connection.execute(sql_code)

In [ ]:
df_from_database = pd.read_sql(sql_code, sql_connection)

In [ ]:
df_from_database

In [ ]:
# When making changes to the database in the form of making updates, deletions or inserts, you need to commit the changes.
# creating an engine with attributes provided
sql_engine2 = mysql_engine()
# establishing connection with the database. Need to use raw_connection()
sql_connection2 = sql_engine2.raw_connection()

# cursor in order to write sql queries with changes that are being made to the database. Not just select and extract keywords.
cursor2 = sql_connection2.cursor()
cursor2.execute("USE wsb_data; DROP TABLE test_tb, test_tb2, test_tb3;")
cursor2.close()
sql_connection2.commit()

## Use this part below to download the comments from reddit with specific subreddit

In [ ]:
from pmaw import PushshiftAPI
api = PushshiftAPI()

In [ ]:
dt.datetime.now()

In [ ]:
days_delta = dt.timedelta(days=90)

In [ ]:
start_date = dt.datetime.now() - days_delta
start_date = int(start_date.timestamp())
start_date

In [ ]:
end_date = dt.datetime.now() - dt.timedelta(days=17)
end_date = int(end_date.timestamp())
end_date

In [ ]:
# Setting subreddit to pull comments from and number of comments to pull in form of a limit value
subreddit="wallstreetbets"
limit=40000

In [ ]:
# Please expect to take some time to download this, i.e. the bigger the number like 100,000 then expect like 10 minutes download time due to reddit api 100 comments per request limit.
comments_data = api.search_comments(subreddit=subreddit, limit=limit, before=end_date, after=start_date)

print(f'Retrieved {len(comments_data)} comments from Pushshift')

In [ ]:
# Here stores the downloaded comments into a dataframe.
full_df = pd.DataFrame(comments_data)

In [ ]:
full_df

In [ ]:
# Saving the downloaded data into a csv file to store the data.
# you can change the name of the file to whatever you want.
full_df.to_csv('./wsb_comments4.csv', header=True, index=False, columns=list(full_df.axes[1]))

In [ ]:
# importing a csv file to put in the database
df_put = pd.read_csv("wsb_comments4.csv")

In [ ]:
df_put

In [ ]:
# Saving the data to the database
df_put = df_put[['author', 'body', 'subreddit', 'subreddit_name_prefixed', 'permalink', 'created_utc']]

In [ ]:
df_put['created_utc'] = df_put['created_utc'].apply(lambda x: utc_to_date_obj(x))

In [ ]:
df_put.rename(columns={"created_utc":"timestamp"}, inplace=True)

In [ ]:
cols = df_put.columns
list(cols)

In [ ]:
df_to_mysql(df_put, mysql_engine(), db_tbl_name)

In [ ]:
# Extract the data from the MySQL database
# SQL Code to run
sql_code = text("""
select * 
from wsb_data.wsb_comments;
""")

# creating an engine with attributes provided
sql_engine = mysql_engine()
# establishing connection with the database
sql_connection = sql_engine.connect()
# Executing the sql code.
extracted_data = sql_connection.execute(sql_code)

In [ ]:
all_comments_df = pd.DataFrame(extracted_data)

In [ ]:
# Preprocessing Data to put in correct from in the dataframe

In [ ]:
def drop_column(column_delete, df):
    df.drop(labels=[column_delete], axis=1, inplace=True)
    return df
all_comments_df = drop_column(int(0), all_comments_df)

In [ ]:
# Renaming the columns
all_comments_df.columns = ['author',
 'body',
 'subreddit',
 'subreddit_name_prefixed',
 'permalink',
 'timestamp']

In [ ]:
all_comments_df

In [ ]:
# lets check for duplicate comments and drop them if found.
check_dupes = all_comments_df.loc[all_comments_df.duplicated(keep=False)]

In [ ]:
check_dupes.loc[check_dupes['body'] == "Its only insulting to white liberals."]

In [ ]:
# keeping the first occurence of duplicates and dropping the subsequent occurence of them rows
def drop_dupes(df):
    indicies_of_rows = df.loc[df.duplicated(keep='last')].index
    indicies_of_rows = list(indicies_of_rows)
    df.drop(labels=indicies_of_rows, axis=0, inplace=True)
    return df
all_comments_df = drop_dupes(all_comments_df)

In [ ]:
# dropping rows where the author and comment body has been deleted
def drop_author_body_empty(df):
    rows_empty = df.loc[(df['author']=="[deleted]") | df['body']=="[deleted]"].index
    rows_empty = list(rows_empty)
    df.drop(labels=rows_empty, axis=0, inplace=True)
    return df
all_comments_df = drop_author_body_empty(all_comments_df)

In [ ]:
# As we can see now we don't have duplicated rows
all_comments_df.loc[all_comments_df.duplicated()]

In [ ]:
# checking for empty strings/NaNs or Nulls
all_comments_df.loc[(all_comments_df['body'] == 'None') | (pd.isna(all_comments_df['body']))]

In [ ]:
# Deleting empty rows
def delete_empty(df):
    rows_to_del = df.loc[(df['body'] == 'None') | (pd.isna(df['body']))].index
    df.drop(labels=rows_to_del, axis=0, inplace=True)
    return df
all_comments_df = delete_empty(all_comments_df)

In [ ]:
all_comments_df = all_comments_df.reset_index(drop=True)
all_comments_df

In [ ]:
# Goal1: Extracting from each comment ticker mentions. DONE!

In [ ]:
# Creating a NLP Pipeline.
import spacy

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
txt = all_comments_df['body'][4]

In [ ]:
txt

In [ ]:
doc = nlp(txt)

In [ ]:
# lets take a look what the model has found.


In [ ]:
entities = []
labels = []
position_start = []
position_end = []

for ent in doc.ents:
    entities.append(ent)
    labels.append(ent.label_)
    position_start.append(ent.start_char)
    position_end.append(ent.end_char)

df = pd.DataFrame({"Entities":entities,"Labels":labels, "Position_start":position_start, "Position_end":position_end})

In [ ]:
doc

In [ ]:
df

In [ ]:
# lets create a function that will look at every comment and extract everything useful from each comment.
def extract_entities(text_string, BLACKLIST=[]):
    # we are passing BLACKLIST variable to exclude arbitrary values you want
    doc = nlp(text_string)
    org_list = []
    for entity in doc.ents:
        # Here we check if the picked up entity is the Organization and whether it is also not in our blacklist variable
        if entity.label_ == 'ORG' and entity.text.lower() not in BLACKLIST:
            org_list.append(str(entity.text).upper())
    # if organization is identified more than once it will appear multiple times in list
    # we use set() to remove duplicates then convert back to list
    # we also do this in order to prevent users to spam the same stock ticker to increase number of mentions for that ticker, hence one comment = one ticker mention of that company
    org_list = list(set(org_list))
    return org_list

In [ ]:
all_comments_df['mentions'] = all_comments_df['body'].apply(lambda x: extract_entities(x))

In [ ]:
all_comments_df.head(50)

In [ ]:
# Creating a Frequency Table
all_orgs = all_comments_df['mentions'].to_list()

In [ ]:
# flattening out the 2d array into 1d array
from itertools import chain

In [ ]:
all_orgs = list(chain.from_iterable(all_orgs))

In [ ]:
len(all_orgs)

In [ ]:
from collections import Counter

In [ ]:
org_freq = Counter(all_orgs)

In [ ]:
org_freq = dict(org_freq)

In [ ]:
freq_table = pd.DataFrame.from_dict(org_freq, orient='index')

In [ ]:
freq_table.rename(columns={0:"Frequency"}, inplace=True)

In [ ]:
freq_table = freq_table.sort_values(by="Frequency", ascending=False)

In [ ]:
freq_table

In [ ]:
# Getting all US stock tickers and company names
import io
from get_all_tickers import get_tickers as gt

In [ ]:
url_of_tickers = "http://ftp.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt"
tickers = requests.get(url_of_tickers)

data_of_tickers = tickers.text
data_content = tickers.content

data_df = pd.read_csv(io.StringIO(data_content.decode("utf-8")), sep="|")
data_df.drop(labels=[11853], axis=0, inplace=True)

In [ ]:
data_df[data_df['Symbol'] == 'VIAC']

In [ ]:
def keep_company_name(txt):
    array = txt.split()
    company_name = array[0].upper()
    return company_name

In [ ]:
data_df['Company_name'] = data_df['Security Name'].apply(lambda x: keep_company_name(x))

In [ ]:
data_df[data_df['Company_name']=='VIACOMCBS']

In [ ]:
# Creaating a dictionary
stock_tickers = data_df['Symbol'].to_list()

In [ ]:
company_names = data_df['Company_name'].to_list()

In [ ]:
# Dictionary of company names and their respective tickers
dict_stock_translator = {}

In [ ]:
for i in range(0,len(company_names)):
    dict_stock_translator[company_names[i]] = dict_stock_translator.get(company_names[i], [])
    dict_stock_translator[company_names[i]].append(stock_tickers[i])

In [ ]:
# tickers to company name
tickers_company_dict = {}

In [ ]:
# Applying the conversion to have only stock tickers present.
for key, value in dict_stock_translator.items():
    for val in value:
        tickers_company_dict[val] = key

In [ ]:
tickers_company_dict['VIAC']

In [ ]:
copy_table = freq_table.copy()

In [ ]:
copy_table = copy_table.reset_index()

In [ ]:
def name_change(txt):
    try:
        txt = tickers_company_dict[txt]
    except KeyError:
        pass
    return txt

In [ ]:
copy_table['new_index'] = copy_table['index'].apply(lambda x: name_change(x))

In [ ]:
copy_table.head(50)

In [ ]:
copy_table.loc[copy_table['new_index'] == "NVIDIA"]

In [ ]:
copy_table.dtypes

In [ ]:
# Grouping by function to get the final frequency table
copy_table

In [ ]:
freq_table_final = copy_table.copy()

In [ ]:
freq_table_final.drop(labels=['index'], axis=1, inplace=True)

In [ ]:
freq_table_final['new_index'] = freq_table_final['new_index'].astype(str)

In [ ]:
freq_table_final = freq_table_final.groupby("new_index")['Frequency'].sum().reset_index()

In [ ]:
freq_table_final = freq_table_final.sort_values(by='Frequency', ascending=False)

In [ ]:
freq_table_final = freq_table_final.reset_index(drop=True)
freq_table_final

In [ ]:
# Last filtering. Only keeping the financial stocks ONLY!
freq_table_final = freq_table_final.loc[freq_table_final['new_index'].isin(list(dict_stock_translator))].reset_index(drop=True)

In [ ]:
# Visualising via Matplotlib and Plotly.
import matplotlib.pyplot as plt
import plotly

In [ ]:
# Top 20
x_vals = freq_table_final['new_index'].tolist()[0:20]
y_vals = freq_table_final['Frequency'].tolist()[0:20]

## Stock Mentions from all the comments provided in a dataframe

In [ ]:
plt.figure(figsize=(25,12))
plt.bar(x_vals, y_vals, label='Stock Mentions')
# plt.axis([0, -1, 0, 500])
plt.ylabel("Number of Mentions", fontsize=16)
plt.xlabel("Company Stock", fontsize=16)
plt.xticks(fontsize=16, rotation=45)
plt.yticks(fontsize=16)
plt.legend(loc="upper center", fontsize=16)
plt.show()

In [ ]:
# Function to search for specific company ticker and display it
def display_ticker_mentions(ticker=None):
    if (type(ticker) == int):
        return
    elif (len(ticker) < 5):
        try:
            company_name = tickers_company_dict[ticker]
        except KeyError:
            return "Make Sure it is a real ticker"
    else:
        company_name = ticker.upper()
    
    company_name = ticker.upper()
    company_data = freq_table_final.loc[freq_table_final['new_index'] == company_name]['Frequency'][0]
    
    # plotting data
    x_pos = [0.5]
    y_vals = company_data
    
    plt.figure(figsize=(25,12))
    plt.bar(x_pos, y_vals, label='Stock Mentions', width=0.1)
    plt.ylabel("Number of Mentions", fontsize=16)
    plt.xlabel("Company Stock", fontsize=16)
    
    plt.axis([0,1,0, company_data + 50])
    
    plt.xticks(x_pos, [company_name], fontsize=16, rotation=45)
    plt.yticks(fontsize=16)
    
    
    plt.legend(loc="upper center", fontsize=16)
    plt.show()
    
display_ticker_mentions(ticker="GAMESTOP")

In [ ]:
# Goal2: Classification of sentiment of the comment/sentence.
# Starting SPARK NLP Preproccessing the text data
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp

In [ ]:
# Creating a Spark-NLP Data Pipeline
spark = sparknlp.start()

In [ ]:
clean_untouched_df = all_comments_df.copy()
clean_untouched_df = clean_untouched_df[['body']]
clean_untouched_df = clean_untouched_df['body'].tolist()

In [ ]:
comments_array_2d = np.array(clean_untouched_df)
comments_array_2d = comments_array_2d.reshape(-1, 1)
comments_array_2d = comments_array_2d.tolist()

In [ ]:
 spark_comments_df = spark.createDataFrame(comments_array_2d).toDF('text')

In [ ]:
# Using my own spark nlp pipeline
# Using 

# initial stage of the preprocessing pipeline. needed to do all the below transformations.
document = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

# Breaks down sentence into list of words i.e. "Nike is better than Adidas" -> ['Nike', 'is', 'better', 'than', 'Adidas']
token = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

# Removes all dirty characters from text following a regex pattern and transforms words based on a provided dictionary
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normal")

# The actual sentiment pretrained model I am using to classifiy the comments
vivekn =  ViveknSentimentModel.pretrained() \
    .setInputCols(["document", "normal"]) \
    .setOutputCol("result_sentiment")

# Once we have our NLP pipeline ready to go, we might want to use our annotation results somewhere else where it is easy to use. The Finisher outputs annotation(s) values into a string.
finisher = Finisher() \
    .setInputCols(["result_sentiment"]) \
    .setOutputCols("final_sentiment")

pipeline = Pipeline().setStages([document, token, normalizer, vivekn, finisher])

pipelineModel = pipeline.fit(spark_comments_df)
result = pipelineModel.transform(spark_comments_df)

# result.select("final_sentiment").show(truncate=False).

In [ ]:
# Transforming the spark nlp df to a pandas df
df_sentiment_comments = result.toPandas()

In [ ]:
# DataFrame with all the comments and their respective positive/neutral/negative sentiment for each comment
df_sentiment_comments['final_sentiment'] = df_sentiment_comments['final_sentiment'].astype(str)
df_sentiment_comments['final_sentiment'][0]

In [ ]:
# Overall Sentiment of all the comments provided:
df_overall_sentiment = df_sentiment_comments['final_sentiment'].value_counts()
df_overall_sentiment = df_overall_sentiment.reset_index()

In [ ]:
# Overall Sentiment of the comments
# As we can see we have more negative than positive comments
df_overall_sentiment

In [ ]:
# Plot a histogram of the sentiment data
plt.figure(figsize=(14,6))
neg_comments = df_sentiment_comments["final_sentiment"][df_sentiment_comments["final_sentiment"] == "['negative']"]
pos_comments = df_sentiment_comments["final_sentiment"][df_sentiment_comments["final_sentiment"] == "['positive']"]
neutral_comments = df_sentiment_comments["final_sentiment"][df_sentiment_comments["final_sentiment"] == "['na']"]
unidentified_comments = df_sentiment_comments["final_sentiment"][df_sentiment_comments["final_sentiment"] == "[]"]
plt.hist([
         neg_comments,
         pos_comments,
         neutral_comments,
         unidentified_comments,
    ],
     width=0.3,
     label=["negative", "positive", "neutral", "na"])
plt.legend()
plt.title("Sentiment of comments")

plt.xticks([0,1,2,3],['negative', 'positive', 'neutral', 'na'])
plt.xlabel("Comment sentiment")
plt.ylabel("# of comments")
plt.show()